In [12]:
import numpy as np
import pandas as pd
import seaborn  as sns
import matplotlib as plt
from datetime import datetime as dt 


In [13]:
df = pd.read_excel("dental_clinic.xlsx")


In [14]:
df.describe()

,Hasta ID,Miktar
count,2085.000000,2085.000000
mean,2867.572182,746.863751
std,1583.850152,1296.312898
min,68.000000,1.000000
25%,1394.000000,200.000000
50%,3221.000000,350.000000
75%,4225.000000,800.000000
max,5197.000000,19000.000000


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2085 entries, 0 to 2084
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Hasta ID     2085 non-null   int64  
 1   Miktar       2085 non-null   float64
 2   Para Birimi  2085 non-null   object 
 3   Tarih        2085 non-null   object 
 4   Ödeme Şekli  2085 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 81.6+ KB


In [16]:
df["Hasta ID"].nunique()

1177

In [17]:
df["Hasta ID"].value_counts()

2098    20
2928    14
3868    12
937     11
288     11
        ..
3369     1
3392     1
3396     1
3405     1
5197     1
Name: Hasta ID, Length: 1177, dtype: int64

In [18]:
df[df["Hasta ID"] == 2928]

,Hasta ID,Miktar,Para Birimi,Tarih,Ödeme Şekli
715,2928,250.0,TRY,02 Mart 2021,Kredi Kartı
729,2928,500.0,TRY,10 Mart 2021,Kredi Kartı
740,2928,250.0,TRY,19 Mart 2021,Kredi Kartı
834,2928,2000.0,TRY,08 Mayıs 2021,Nakit
904,2928,500.0,TRY,07 Haziran 2021,Havale / EFT
955,2928,500.0,TRY,07 Temmuz 2021,Havale / EFT
1005,2928,500.0,TRY,08 Ağustos 2021,Havale / EFT
1070,2928,500.0,TRY,07 Eylül 2021,Havale / EFT
1126,2928,500.0,TRY,07 Ekim 2021,Havale / EFT
1185,2928,500.0,TRY,03 Kasım 2021,Havale / EFT


In [20]:
df["Miktar"].sum()

1557210.92

In [21]:
df["Tarih"]

0         17 Mart 2019
1         13 Ekim 2019
2         13 Ekim 2019
3        01 Şubat 2020
4         18 Mart 2020
             ...      
2080    20 Temmuz 2022
2081    20 Temmuz 2022
2082    20 Temmuz 2022
2083    20 Temmuz 2022
2084    21 Temmuz 2022
Name: Tarih, Length: 2085, dtype: object

In [22]:
df["Tarih"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2085 entries, 0 to 2084
Series name: Tarih
Non-Null Count  Dtype 
--------------  ----- 
2085 non-null   object
dtypes: object(1)
memory usage: 16.4+ KB


In [23]:
df["Tarih"].replace(["Ocak","Şubat","Mart","Nisan","Mayıs","Haziran","Temmuz","Ağustos","Eylül","Ekim","Kasım","Aralık"],["January","February","March","April","May","June","July","August","September","October","November","December"],regex=True, inplace = True)
df["Tarih"]

0          17 March 2019
1        13 October 2019
2        13 October 2019
3       01 February 2020
4          18 March 2020
              ...       
2080        20 July 2022
2081        20 July 2022
2082        20 July 2022
2083        20 July 2022
2084        21 July 2022
Name: Tarih, Length: 2085, dtype: object

In [24]:
dt.strptime("17 March 2017", "%d %B %Y")

datetime.datetime(2017, 3, 17, 0, 0)

In [25]:
df["Tarih"] = df["Tarih"].apply(lambda x : dt.strptime(x, "%d %B %Y"))
df["Tarih"]

0      2019-03-17
1      2019-10-13
2      2019-10-13
3      2020-02-01
4      2020-03-18
          ...    
2080   2022-07-20
2081   2022-07-20
2082   2022-07-20
2083   2022-07-20
2084   2022-07-21
Name: Tarih, Length: 2085, dtype: datetime64[ns]

In [27]:
rfm = df.groupby("Hasta ID").agg({"Tarih" : lambda date : (dt.today() - date.max()).days, "Hasta ID" : lambda num : len(num), "Miktar" : lambda total : sum(total)})

In [28]:
rfm.columns = ["Recency","Frequency","Monetary"]
rfm



,Recency,Frequency,Monetary
Hasta ID,,,
68,872,1,175.0
70,152,5,5300.0
72,128,5,9519.0
77,869,1,100.0
82,837,3,1350.0
...,...,...,...
5181,49,1,200.0
5182,49,1,415.0
5187,48,1,415.0


In [29]:
rfm["RecencyScore"] = pd.qcut(rfm["Recency"],5, labels=[5,4,3,2,1])

In [30]:
np.arange(7,11,2)

array([7, 9])

In [32]:
rfm["FrequencyScore"] = rfm["Frequency"].replace([np.arange(2,5),np.arange(5,8),np.arange(8,10),np.arange(10,100)],[2,3,4,5])

In [33]:
rfm["MonetaryScore"] = pd.qcut(rfm["Monetary"],5,labels=[1,2,3,4,5])

In [34]:
rfm

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore
Hasta ID,,,,,,
68,872,1,175.0,1,1,2
70,152,5,5300.0,4,3,5
72,128,5,9519.0,4,3,5
77,869,1,100.0,1,1,1
82,837,3,1350.0,1,2,4
...,...,...,...,...,...,...
5181,49,1,200.0,5,1,2
5182,49,1,415.0,5,1,3
5187,48,1,415.0,5,1,3


In [35]:
rfm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1177 entries, 68 to 5197
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Recency         1177 non-null   int64   
 1   Frequency       1177 non-null   int64   
 2   Monetary        1177 non-null   float64 
 3   RecencyScore    1177 non-null   category
 4   FrequencyScore  1177 non-null   int64   
 5   MonetaryScore   1177 non-null   category
dtypes: category(2), float64(1), int64(3)
memory usage: 48.7 KB


In [36]:
rfm[["RecencyScore","MonetaryScore"]] = rfm[["RecencyScore","MonetaryScore"]].astype("int64")

In [37]:
rfm["Patient_Point"] = ((rfm["RecencyScore"]+rfm["FrequencyScore"]+rfm["MonetaryScore"]) / 3)

In [38]:
rfm

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,Patient_Point
Hasta ID,,,,,,,
68,872,1,175.0,1,1,2,1.333333
70,152,5,5300.0,4,3,5,4.000000
72,128,5,9519.0,4,3,5,4.000000
77,869,1,100.0,1,1,1,1.000000
82,837,3,1350.0,1,2,4,2.333333
...,...,...,...,...,...,...,...
5181,49,1,200.0,5,1,2,2.666667
5182,49,1,415.0,5,1,3,3.000000
5187,48,1,415.0,5,1,3,3.000000


In [39]:
rfm["RF-M"] = round(((rfm["RecencyScore"] + rfm["FrequencyScore"]) / 2).astype("int")).astype ("str") + rfm["MonetaryScore"].astype("str")
rfm

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,Patient_Point,RF-M
Hasta ID,,,,,,,,
68,872,1,175.0,1,1,2,1.333333,12
70,152,5,5300.0,4,3,5,4.000000,35
72,128,5,9519.0,4,3,5,4.000000,35
77,869,1,100.0,1,1,1,1.000000,11
82,837,3,1350.0,1,2,4,2.333333,14
...,...,...,...,...,...,...,...,...
5181,49,1,200.0,5,1,2,2.666667,32
5182,49,1,415.0,5,1,3,3.000000,33
5187,48,1,415.0,5,1,3,3.000000,33


In [41]:
seg_map = {"[1-2][1-2]":"hibernating", "[1-2][3-4]":"at_risk(expensive_treatment_patient)", "[1-2]5":"can't_loose(very_important_patient)", "3[1-2]":"about_to_sleep", "33":"need_attention", "[3-4][4-5]":"potential_loyal_customers", "[4-5][1-2]":"cheap_work_patients", "[4-5][3-4]":"loyal_customers", "[4-5][4-5]":"champions" }

In [42]:
seg_map

{'[1-2][1-2]': 'hibernating',
 '[1-2][3-4]': 'at_risk(expensive_treatment_patient)',
 '[1-2]5': "can't_loose(very_important_patient)",
 '3[1-2]': 'about_to_sleep',
 '33': 'need_attention',
 '[3-4][4-5]': 'potential_loyal_customers',
 '[4-5][1-2]': 'cheap_work_patients',
 '[4-5][3-4]': 'loyal_customers',
 '[4-5][4-5]': 'champions'}

In [44]:
rfm["segment"] = rfm["RF-M"].replace(seg_map,regex=True)

In [45]:
rfm

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,Patient_Point,RF-M,segment
Hasta ID,,,,,,,,,
68,872,1,175.0,1,1,2,1.333333,12,hibernating
70,152,5,5300.0,4,3,5,4.000000,35,potential_loyal_customers
72,128,5,9519.0,4,3,5,4.000000,35,potential_loyal_customers
77,869,1,100.0,1,1,1,1.000000,11,hibernating
82,837,3,1350.0,1,2,4,2.333333,14,at_risk(expensive_treatment_patient)
...,...,...,...,...,...,...,...,...,...
5181,49,1,200.0,5,1,2,2.666667,32,about_to_sleep
5182,49,1,415.0,5,1,3,3.000000,33,need_attention
5187,48,1,415.0,5,1,3,3.000000,33,need_attention
